In [37]:
import pandas as pd
import os
import time
import logging
from datetime import date

In [38]:
# # 获取脚本的当前路径，避免计划执行时路径出错
# home_dir = os.path.dirname(os.path.realpath(__file__))
# # 更换workding directory
# working_directory = home_dir
# os.chdir(working_directory)

In [39]:
logging.basicConfig(level=logging.INFO, format=' %(asctime)s - %(levelname)s - %(message)s')

In [40]:
folder_path = '../shopee_product_translation/'
tw_site_product_list = pd.read_excel(os.path.join(folder_path, 'tengus_tw_product_list.xlsx'))
# tw_site_product_list = pd.read_excel(os.path.join(folder_path, 'translated_product_2019_10_19.xlsx'))
# tw_site_product_list = tw_site_product_list[tw_site_product_list['是否翻译'] == 'no']
# tw_site_product_list.head(5)

In [41]:
# tw_site_product_list.columns

In [42]:
# tw_site# 获取脚本的当前路径，避免计划执行时路径出错
# home_dir = os.path.dirname(os.path.realpath(__file__))
# # 更换workding directory
# working_directory = home_dir
# os.chdir(working_directory)_product_list = tw_site_product_list[tw_site_product_list['产品id'] == 2496637042]

In [43]:
# top_cat_in_tw = tw_site_product_list[['分类ID', '产品id']].groupby('分类ID').agg({'产品id': 'nunique'}).reset_index().sort_values(by='产品id', ascending=False).reset_index().head(5)
# top_cat_in_tw

In [44]:
# top_cat_id = top_cat_in_tw['分类ID'][0]
# top_cat_id

In [45]:
output_excel_columns = [
                        '变种名称',
                        '变种属性名称一',
                        '变种属性名称二',
                        '变种属性值一',
                        '变种属性值二',
                        '*价格（必填）',
                        '促销价',
                        '折扣活动ID',
                        '*库存（必填）',
                        '*重量(kg)(必填）',
                        '*主图（URL）地址（必填）',
                        '附图1',
                        '附图2',
                        '附图3',
                        '附图4',
                        '附图5',
                        '附图6',
                        '附图7',
                        '附图8',
                        '变种图',
                        '长(cm)',
                        '宽(cm)',
                        '高(cm)',
                        '发货期',
                        '来源URL',
                        '尺码图'
]

In [46]:
tw_site_product_list_selected_columns = ['分类ID', '产品属性', 'Parent SKU', '产品标题', '产品描述', 'sku', '变种名称', '变种属性名称一',
       '变种属性名称二', '变种属性值一', '变种属性值二', '价格', '促销价', '折扣活动ID', '库存', '重量', '主图（URL）地址', '附图1',
       '附图2', '附图3', '附图4', '附图5', '附图6', '附图7', '附图8地址', '变种图', '长（cm）',
       '宽（cm）', '高（cm）', '发货期', '来源URL', '尺码图']

In [47]:
tw_site_product_list_with_selected_columns = tw_site_product_list[tw_site_product_list_selected_columns]
# tw_site_product_list_with_selected_columns.head(5)

In [48]:
# tw_site_product_list_with_selected_columns = tw_site_product_list_with_selected_columns[tw_site_product_list_with_selected_columns['分类ID'] == top_cat_id].reset_index()
# tw_site_product_list_with_selected_columns

In [49]:
# 把1688的来源url拆出来，把productid填充为parent sku
tw_site_product_list_with_selected_columns['Parent SKU'] = tw_site_product_list_with_selected_columns['来源URL'].str.split("/", n = 4, expand = True)[4].str.split(".", n = 1, expand=True)[0]
# tw_site_product_list_with_selected_columns.head(5)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [50]:
# remove emoji
import re

emoji_pattern = re.compile(
                u'['
                u'\U0001F300-\U0001F64F'
                u'\U0001F680-\U0001F6FF'
                u'\u2600-\u2B55'
                u'\u23cf'
                u'\u23e9'
                u'\u231a'
                u'\u3030'
                u'\ufe0f'
                u"\U0001F600-\U0001F64F"  # emoticons
                u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                u'\U00010000-\U0010ffff'
                u'\U0001F1E0-\U0001F1FF'  # flags (iOS)
                u'\U00002702-\U000027B0]+', flags=re.UNICODE)

def remove_emoji(text):
    return emoji_pattern.sub(r'', text)

# remove_emoji('ins爆款 🥕胡蘿卜🥕寶寶針織竹節棉爬服 嬰兒兔裝 嬰兒連身衣 寶寶包屁衣連身衣 中間開襟包屁衣 寶寶連體衣哈衣爬服')

In [51]:
tw_site_product_list_with_selected_columns['产品标题'] = [remove_emoji(x) for x in tw_site_product_list_with_selected_columns['产品标题']]
tw_site_product_list_with_selected_columns['产品描述'] = [remove_emoji(x) for x in tw_site_product_list_with_selected_columns['产品描述']]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [52]:
tw_site_product_list_with_selected_columns['长（cm）'] = tw_site_product_list_with_selected_columns['长（cm）'].fillna(18)
tw_site_product_list_with_selected_columns['宽（cm）'] = tw_site_product_list_with_selected_columns['宽（cm）'].fillna(13)
tw_site_product_list_with_selected_columns['高（cm）'] = tw_site_product_list_with_selected_columns['高（cm）'].fillna(5)
tw_site_product_list_with_selected_columns['发货期'] = tw_site_product_list_with_selected_columns['发货期'].fillna(2)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

In [53]:
set(tw_site_product_list_with_selected_columns['变种属性名称一'].tolist())

{'规格（长*宽cm）', '颜色'}

In [54]:
set(tw_site_product_list_with_selected_columns['变种属性名称二'].tolist())

{nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan

In [55]:
set(tw_site_product_list_with_selected_columns['变种属性值一'].tolist())

{'10号',
 '10号小熊吃蜂蜜',
 '10号蓝色小西服',
 '10号黄色小狗',
 '10蓝色圆点大象',
 '11号',
 '11号小熊开汽车',
 '11号绿色狮子',
 '11号蓝色小兔兔',
 '11号蓝色米奇',
 '12号粉色小花朵',
 '12号蓝色西服',
 '12号长颈鹿',
 '13号',
 '13号猴子',
 '13号绿色青蛙',
 '13号黄色小羊',
 '14号',
 '14号彩色绵羊',
 '14号绿色小鸟',
 '14号蓝色小毛驴',
 '15号',
 '15号小熊开汽车',
 '15号熊猫',
 '15号红色大草莓',
 '16号唱歌青蛙',
 '16号红色摩托车',
 '16号草莓',
 '17号大象',
 '17号小蜜蜂',
 '17号粉色兔子',
 '18号',
 '18号小汽车',
 '19号',
 '19号小熊',
 '19号粉色羊驼',
 '1号',
 '1号大草莓',
 '1号粉色小兔子',
 '1号黄色长颈鹿猴子',
 '20号小熊打篮球',
 '20号紫色昆虫',
 '21号粉色小猪猪',
 '22号长颈鹿',
 '23号',
 '24号',
 '2号',
 '2号大黄鸭',
 '2号粉色云朵',
 '2号粉色表情兔',
 '2号蓝色白色',
 '2号黄花大象',
 '3号',
 '3号动物乐园',
 '3号浅绿三角形',
 '3号粉色花朵',
 '3号蓝色小奶牛',
 '4号猴子',
 '4号粉色兔子',
 '4号粉色圆点',
 '4号绿色恐龙',
 '4号肉粉花瓣',
 '5号',
 '5号卡其蝴蝶结',
 '5号快乐小马',
 '5号粉色大猪',
 '5号绿色小青蛙',
 '6号',
 '6号粉色皇冠',
 '6号红色大汽车',
 '6号红色长颈鹿',
 '6号蓝色汽车',
 '6号青蛙',
 '7号',
 '7号小螃蟹',
 '7号绿色猴子',
 '7号蓝色小汽车',
 '7号黄底五角星',
 '7号黄色皇冠',
 '8号',
 '8号小汽车',
 '8号彩色云朵',
 '8号快乐小马',
 '8号绿色螃蟹',
 '9号小狮子',
 '9号红色小熊开车',
 '9号蓝色猴子',
 '9号蓝色精灵',
 '兔子月亮船',
 '动物园',
 '印花款兰色',
 '印花款粉色',
 '印花款颜色均配',


In [56]:
set(tw_site_product_list_with_selected_columns['变种属性值二'].tolist())

{nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan

In [57]:
# translate_currency = 458.61
# profit_rate = 1.2
# tw_site_product_list_with_selected_columns['价格'] = tw_site_product_list_with_selected_columns['价格'] * translate_currency / profit_rate

In [58]:
# tw_site_product_list_with_selected_columns.head(5)

In [59]:
from googletrans import Translator

In [60]:
translator = Translator(service_urls=[
      'translate.google.com',
      'translate.google.co.kr',
    ])
# translator.translate('韩国', dest='ko').text

In [61]:
# translations = translator.translate(['The quick brown fox', 'jumps over', 'the lazy dog'], dest='ko')
# for translation in translations:
#     print(translation.origin, ' -> ', translation.text)

In [62]:
translate_columns = ['产品标题', '产品描述', '变种名称', '变种属性名称一', '变种属性名称二', '变种属性值一', '变种属性值二']
# translate_columns = ['产品标题']
# translate_columns = ['产品描述', '变种名称', '变种属性名称一', '变种属性名称二', '变种属性值一', '变种属性值二']

In [63]:
# 把空值填充为string格式
tw_site_product_list_with_selected_columns[translate_columns] = tw_site_product_list_with_selected_columns[translate_columns].fillna(value='不需翻译')

In [64]:
# 把variation_name做一些删减
variation_list = set(list(tw_site_product_list_with_selected_columns['变种属性名称一'].unique()) + list(tw_site_product_list_with_selected_columns['变种属性名称二'].unique()))
logging.info('variation list: ' + str(variation_list))

 2019-11-24 23:19:42,244 - INFO - variation list: {'规格（长*宽cm）', '颜色', '不需翻译'}


In [65]:
variation_rename = {'适合身高': '身高',
                    '童袜尺码': '尺码',
                    '不需翻译': 'not set',
                    '73CM': '73cm',
                    '100cm 少量现货': '100cm',
                    '100CM建议体重32斤内': '100cm',
                    '110CM建议体重36斤内': '110cm',
                    '120CM建议体重40斤内': '120cm',
                    '73CM建议体重18斤内': '73cm',
                    '80CM建议体重23斤内': '80cm',
                    '90CM建议体重28斤内': '90cm',
                    '编码': 'no size',
                    'XXL码（建议身高130-150cm）':'XXL',
                    'L码 （建议身高100-100cm）':'L ',
                    'XL码(建议身高115-130cm)':'XL',
                    '20/21（实际内长12.4cm）':'20/21',
                    '22/23（实际内长13.6cm）':'22/23',
                    '24/25（实际内长14.7cm）':'24/25',
                    '26/27（实际内长15.9CM）':'26/27',
                    '28/29（实际内长17cm）':'28/29',
                    '袋装（加大 6-18个月）':'6-18月',
                    '袋装（常规 0-12个月）':'0-12月',
                    'M码（2-4岁 防滑底）':'M',
                    'S码（0-2岁 防滑底）':'S',
                    'M（1-3岁 防滑底）':'M',
                    'M（2-4岁 防滑底）':'M',
                    'S（0-1岁 防滑底）':'S',
                    'S（0-2岁 防滑底）':'S',
                    'S (6-12)个月':'S ',
                    'S （6-12个月）':'S ',
                    'XS( 0-6个)月':'XS',
                    '12-24个月闭档':'12-24月',
                    '32*12.5cm':'32*12.5cm',
                    'L码24-36个月':'L',
                    'XL码 9-12岁':'XL',
                    'XL码（5-8岁）':'XL',
                    'XS（0-6个月）':'XS',
                    '6-12个月闭档':'6-12月',
                    'L码 7-10岁':'L',
                    'L码（3-5岁）':'L',
                    'L码（7-9岁）':'L',
                    'M （1-3）岁':'M ',
                    'M码（1-3岁）':'M',
                    'M码（4-6岁）':'M',
                    'S码（0-1岁）':'S',
                    'S码（1-3岁）':'S',
                    'XL码 5-8岁':'XL',
                    'XL码9-12岁':'XL',
                    '（0-1岁）均码':'0-1岁',
                    'L码 3-5岁':'L',
                    'L码 5-8岁':'L',
                    'L（3-5岁）':'L',
                    'M码 1-3岁':'M',
                    'M码 3-5岁':'M',
                    'M码 4-6岁':'M',
                    'M（1-3岁）':'M',
                    'S码 0-1岁':'S',
                    'S码 1-3岁':'S',
                    '（1-3岁）均码':'1-3岁',
                    '12*9CM':'12*9CM',
                    '12-24月':'12-24月',
                    '2-4岁闭档':'2-4岁',
                    '4-6岁闭档':'4-6岁',
                    '6-12个月':'6-12月',
                    '6-18个月':'6-18月',
                    'L码3-5岁':'L',
                    'L码4-6岁':'L',
                    'L码5-8岁':'L',
                    'L码6-8岁':'L',
                    'M码1-3岁':'M',
                    'M码2-4岁':'M',
                    'M码3-5岁':'M',
                    'S码0-1岁':'S',
                    'S码0-2岁':'S',
                    'S码1-3岁':'S',
                    '0-2岁':'0-2岁',
                    '1-3岁':'1-3岁',
                    '11CM':'11CM',
                    '11cm':'11cm',
                    '12CM':'12CM',
                    '12cm':'12cm',
                    '13CM':'13CM',
                    '13cm':'13cm',
                    '14cm':'14cm',
                    '2-4岁':'2-4岁',
                    '28CM':'28CM',
                    '28cm':'28cm',
                    '3-5岁':'3-5岁',
                    '35CM':'35CM',
                    '35cm':'35cm',
                    '4-6岁':'4-6岁',
                    '45CM':'45CM',
                    '45cm':'45cm',
                    '5-8岁':'5-8岁',
                    '均码':'均码'
}
tw_site_product_list_with_selected_columns['变种属性名称一'] = tw_site_product_list_with_selected_columns['变种属性名称一'].apply(lambda x: variation_rename[x] if x in variation_rename else x)
tw_site_product_list_with_selected_columns['变种属性名称二'] = tw_site_product_list_with_selected_columns['变种属性名称二'].apply(lambda x: variation_rename[x] if x in variation_rename else x)
tw_site_product_list_with_selected_columns['变种属性值一'] = tw_site_product_list_with_selected_columns['变种属性值一'].apply(lambda x: variation_rename[x] if x in variation_rename else x)
tw_site_product_list_with_selected_columns['变种属性值二'] = tw_site_product_list_with_selected_columns['变种属性值二'].apply(lambda x: variation_rename[x] if x in variation_rename else x)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:97: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:98: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-

In [66]:
# 给变种属性值一、二做适当缩减
# 先把索引列取出来
tw_site_product_list_with_selected_columns['index'] = tw_site_product_list_with_selected_columns.index
tw_site_product_list_with_selected_columns['变种属性值一'] = tw_site_product_list_with_selected_columns['index'].groupby([tw_site_product_list_with_selected_columns['Parent SKU'],tw_site_product_list_with_selected_columns['变种属性值二']]).rank(ascending=1,method='dense')

tw_site_product_list_with_selected_columns['变种属性值一'] = tw_site_product_list_with_selected_columns['变种属性值一'].apply(str)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usin

In [67]:
tw_site_product_list_with_selected_columns = tw_site_product_list_with_selected_columns.drop(['index'], axis=1)

In [68]:
# 价格 折扣价
tw_site_product_list_with_selected_columns['促销价'] = tw_site_product_list_with_selected_columns['价格']
tw_site_product_list_with_selected_columns['价格'] = tw_site_product_list_with_selected_columns['价格'] * 10
tw_site_product_list_with_selected_columns['折扣活动ID'] = 1035824326

# 重量
tw_site_product_list_with_selected_columns['重量'] = 0.3

In [69]:
tw_site_product_list_with_selected_columns.to_excel(os.path.join(folder_path, 'to_be_translated_product.xlsx'), index=False)

In [70]:
# 对variation_value做一些删减
# variation_value_list = set(list(tw_site_product_list_with_selected_columns['变种属性值一'].unique()) + list(tw_site_product_list_with_selected_columns['变种属性值一'].unique()))
# logging.info('variation value list: ' + str(variation_value_list))

In [71]:
# tw_site_product_list_with_selected_columns = tw_site_product_list_with_selected_columns[:5]
# # tw_site_product_list_with_selected_columns

In [72]:
today = date.today()
d1 = today.strftime("%Y_%m_%d")

In [73]:
# for index, row in tw_site_product_list_with_selected_columns.iterrows():
#     try:
#         list_to_translate = []
#         for i, value in enumerate(translate_columns):
#             list_to_translate.append(row[translate_columns[i]])
        
#         translate_language = 'en'
#         translator = Translator(service_urls=['translate.google.com'])
#         list_be_translated = translator.translate(list_to_translate, dest=translate_language, src='zh-CN')
#         # list_be_translated = translator.translate(list_to_translate, dest=translate_language)
#         time.sleep(10)
        
#         i = 0
#         for translate_language in list_be_translated:
#             tw_site_product_list_with_selected_columns.at[index, translate_columns[i]] = translate_language.text
#             i += 1
        
#         if (index + 1) % 5 == 0:
#             logging.info(str(index + 1) + ' rows have been translated.')
            
#         tw_site_product_list_with_selected_columns.to_excel(os.path.join(folder_path, 'translated_product_' + d1 + '.xlsx'), index=False)
#     except Exception as err:
#         logging.warning(str(index + 1) + ' rows have not been translated. Error message: ' + str(err))
#         pass

In [74]:
for index, row in tw_site_product_list_with_selected_columns.iterrows():
    translated = 0
    while translated == 0:
        try:
            list_to_translate = []
            for i, value in enumerate(translate_columns):
                list_to_translate.append(row[translate_columns[i]])

            # translate_language = 'id'
            translate_language = 'th'
            translator = Translator(service_urls=['translate.google.com'])
            list_be_translated = translator.translate(list_to_translate, dest=translate_language, src='zh-CN')
            # list_be_translated = translator.translate(list_to_translate, dest=translate_language)
            time.sleep(10)

            i = 0
            for translate_language in list_be_translated:
                tw_site_product_list_with_selected_columns.at[index, translate_columns[i]] = translate_language.text
                i += 1

            if (index + 1) % 100 == 0:
                logging.info(str(index + 1) + ' rows have been translated.')

            tw_site_product_list_with_selected_columns.to_excel(os.path.join(folder_path, 'translated_product_th_' + d1 + '.xlsx'), index=False)
            translated = 1
        except Exception as err:
            logging.warning(str(index + 1) + ' rows have not been translated. Error message: ' + str(err) + ', try again.')
            pass        

 2019-11-24 23:23:15,258 - WARNING - 14 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: /translate_a/single?client=t&sl=zh-cn&tl=th&hl=th&dt=at&dt=bd&dt=ex&dt=ld&dt=md&dt=qca&dt=rw&dt=rm&dt=ss&dt=t&ie=UTF-8&oe=UTF-8&otf=1&ssel=0&tsel=0&tk=659756.834979&q=%E5%84%BF%E7%AB%A5%E7%BA%B1%E5%B8%83%E9%AB%98%E5%AF%86%E5%BA%A68%E4%BB%B6%E5%A5%97+%E5%A9%B4%E5%84%BF%E5%8F%8C%E5%B1%82%E7%BA%B1%E5%B8%83%E6%B5%B4%E5%B7%BE+%E5%AE%9D%E5%AE%9D%E6%89%8B%E5%B8%95%E5%B0%8F%E6%96%B9%E5%B7%BE8%E4%BB%B6%E5%A5%97%E8%A3%85 (Caused by ProxyError('Cannot connect to proxy.', RemoteDisconnected('Remote end closed connection without response'))), try again.
 2019-11-24 23:23:20,261 - WARNING - 14 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')"))), try again.
 

 2019-11-24 23:25:37,644 - WARNING - 14 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')"))), try again.
 2019-11-24 23:25:42,651 - WARNING - 14 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')"))), try again.
 2019-11-24 23:25:47,661 - WARNING - 14 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')"))), try again.
 2019-11-24 23:25:52,674 - WARNING - 14 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url

 2019-11-24 23:30:43,070 - WARNING - 24 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')"))), try again.
 2019-11-24 23:30:48,070 - WARNING - 24 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')"))), try again.
 2019-11-24 23:30:53,094 - WARNING - 24 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')"))), try again.
 2019-11-24 23:30:58,117 - WARNING - 24 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url

 2019-11-24 23:35:34,338 - WARNING - 33 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')"))), try again.
 2019-11-24 23:35:39,367 - WARNING - 33 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')"))), try again.
 2019-11-24 23:35:44,383 - WARNING - 33 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')"))), try again.
 2019-11-24 23:35:49,396 - WARNING - 33 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url

 2019-11-24 23:39:47,589 - WARNING - 40 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')"))), try again.
 2019-11-24 23:39:52,599 - WARNING - 40 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')"))), try again.
 2019-11-24 23:39:57,647 - WARNING - 40 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')"))), try again.
 2019-11-24 23:40:02,673 - WARNING - 40 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url

 2019-11-25 00:18:35,258 - WARNING - 149 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')"))), try again.
 2019-11-25 00:18:40,273 - WARNING - 149 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')"))), try again.
 2019-11-25 00:18:45,289 - WARNING - 149 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')"))), try again.
 2019-11-25 00:18:50,305 - WARNING - 149 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with

 2019-11-25 00:24:21,180 - WARNING - 161 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')"))), try again.
 2019-11-25 00:24:26,195 - WARNING - 161 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')"))), try again.
 2019-11-25 00:24:31,212 - WARNING - 161 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')"))), try again.
 2019-11-25 00:24:36,227 - WARNING - 161 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with

 2019-11-25 00:29:27,664 - WARNING - 170 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')"))), try again.
 2019-11-25 00:29:32,680 - WARNING - 170 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')"))), try again.
 2019-11-25 00:29:37,695 - WARNING - 170 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')"))), try again.
 2019-11-25 00:29:42,711 - WARNING - 170 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with

 2019-11-25 00:34:16,978 - WARNING - 178 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')"))), try again.
 2019-11-25 00:34:22,008 - WARNING - 178 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')"))), try again.
 2019-11-25 00:34:27,023 - WARNING - 178 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')"))), try again.
 2019-11-25 00:34:32,039 - WARNING - 178 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with

 2019-11-25 00:39:07,992 - WARNING - 186 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')"))), try again.
 2019-11-25 00:39:13,008 - WARNING - 186 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')"))), try again.
 2019-11-25 00:39:18,023 - WARNING - 186 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')"))), try again.
 2019-11-25 00:39:23,039 - WARNING - 186 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with

In [75]:
# 优化结果
output_path = os.path.join(folder_path, 'translated_product_th_' + d1 + '.xlsx')
dianxiaomi_product_list = pd.read_excel(output_path)

dianxiaomi_product_list['产品标题'] = dianxiaomi_product_list['产品标题'].apply(lambda x: x[:100])

variation_name_key_value = {
    'ไม่จำเป็นต้องแปล': 'size',
    'ไม่ได้ตั้งค่า': 'size',
    'เหมาะสำหรับความสูง': 'ความสูง',
    'ข้อมูลจำเพาะ (กว้าง * ยาวซม.)': 'size',
    'tidak diatur': 'size'
}

dianxiaomi_product_list['变种属性名称二'] = dianxiaomi_product_list['变种属性名称二'].apply(lambda x: variation_name_key_value[x] if x in variation_name_key_value else x)

In [76]:
# dianxiaomi_product_list['index'] = dianxiaomi_product_list.index
# dianxiaomi_product_list['变种属性值一'] = dianxiaomi_product_list['index'].groupby([dianxiaomi_product_list['Parent SKU'],dianxiaomi_product_list['变种属性值二']]).rank(ascending=1,method='dense')

# dianxiaomi_product_list['变种属性值一'] = dianxiaomi_product_list['变种属性值一'].apply(str)

In [77]:
dianxiaomi_product_list.to_excel(os.path.join(folder_path, 'translated_product_th_' + d1 + '_edited(1).xlsx'), index=False)

In [78]:
# for translate_column in translate_columns:
#     # tw_site_product_list_with_selected_columns[translate_column] = tw_site_product_list_with_selected_columns[translate_column].fillna(0)
#     # tw_site_product_list_with_selected_columns[translate_column]  = [translator.translate(x, dest=translate_language).text 
#                                                                      # for x in tw_site_product_list_with_selected_columns[translate_column]]
#     for index, row in tw_site_product_list_with_selected_columns.iterrows():
#         try:
#             orginal_text = tw_site_product_list_with_selected_columns.at[index, translate_column]
#             tw_site_product_list_with_selected_columns.at[index, translate_column] = translator.translate(row[translate_column], dest=translate_language).text
#             # print(row[translate_column])
#             logging.info(translate_column + ' - ' + str(index + 1) + ' rows have been translated.')
#             # logging.info(orginal_text + ' -> ' + tw_site_product_list_with_selected_columns.at[index, translate_column])
#             time.sleep(5)
#         except:
#             logging.warning(translate_column + ' - ' + str(index + 1) + ' rows have not been translated.')
#             pass
        
#     # print(tw_site_product_list_with_selected_columns.head(5))
    
#     tw_site_product_list_with_selected_columns.to_excel(os.path.join(folder_path, 'tw_translated_product(1).xlsx'), index=False)
#     logging.info(str(translate_column) + ' is done.')